In [1]:
!pip3 install opencv-contrib-python
# Удалить opencv, оставить только opencv-contrib!


Defaulting to user installation because normal site-packages is not writeable


In [24]:
import cv2
import numpy as np
import os
import glob
import pickle

In [25]:
dir = r'test2'
images = glob.glob(os.path.join(dir, "*.jpg"))
arucoDict = cv2.aruco.DICT_4X4_250
board_size = (5, 7)

In [26]:
print(cv2.__version__)
print(cv2.getBuildInformation())

4.9.0

General configuration for OpenCV 4.9.0 =====================================
  Version control:               4.9.0-dirty

  Extra modules:
    Location (extra):            /io/opencv_contrib/modules
    Version control (extra):     4.9.0

  Platform:
    Timestamp:                   2023-12-31T11:19:02Z
    Host:                        Linux 5.15.0-1053-azure x86_64
    CMake:                       3.28.1
    CMake generator:             Unix Makefiles
    CMake build tool:            /bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):

Генерация CHARUCO:
papper - размеры листа бумаги
squareLength - размеры в см грайни квадрата
Печатать в 1:1 масштабе. Dpi 96

In [27]:



papper = [21.9, 31.4] # Размеры листа в см
squareLength = 4 # Размеры квадрата в см
dpi = 96

boardSize = [int(papper[0] // squareLength),\
             int(papper[1] // squareLength)]

print(f'BoardSize: {boardSize}')
def create_board(boardSize, squareLength, arucoDict, dpi):
    markerLength = squareLength  / 2 # Размеры маркера в 2 раза меньше
    dictionary = cv2.aruco.getPredefinedDictionary(arucoDict)
    board = cv2.aruco.CharucoBoard(boardSize, squareLength, markerLength, dictionary)
    frameSize = [int(np.ceil(papper[0] * dpi / 2.54)),\
                 int(np.ceil(papper[1] * dpi / 2.54))] # Длина в пиксели 96dpi
    print(f'Pixels: {frameSize}')
    image = board.generateImage(frameSize , 1, 1)
    cv2.imwrite("BoardImage.jpg", image)
    
create_board(boardSize, squareLength, arucoDict, dpi)

BoardSize: [5, 7]
Pixels: [828, 1187]


In [28]:
def detect_charuco_corners(frame, boardSize):
    dictionary = cv2.aruco.getPredefinedDictionary(arucoDict)
    board = cv2.aruco.CharucoBoard(boardSize, 0.04, 0.02, dictionary)
    params =  cv2.aruco.DetectorParameters()
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_NONE
    image_copy = frame.copy()
    charucodetector = cv2.aruco.CharucoDetector(board)
    charuco_corners, charuco_ids, marker_corners, marker_ids = charucodetector.detectBoard(image_copy)
    #print(board.getChessboardSize())
    #if charuco_ids is not None and len(charuco_ids) > 0:
        #cv2.aruco.drawDetectedCornersCharuco(image_copy, charuco_corners, charuco_ids, (255, 0, 0))
        #cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
    #calibrate_camera(charuco_corners, board, input_video)
    return image_copy, charuco_corners, charuco_ids
    
    

def read_chessboards(images, boardSize):
    dictionary = cv2.aruco.getPredefinedDictionary(arucoDict)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    
    allCorners = []
    allIds = []

    for img in images:
        print("Processing image {0}".format(img))
        frame = cv2.imread(img)
        rgb = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        _, charuco_corners, charuco_ids = detect_charuco_corners(rgb, boardSize)
        if charuco_corners is not None:
                allCorners.append(charuco_corners)
                allIds.append(charuco_ids)

    imsize = rgb.shape
    return allCorners,allIds,imsize

allCorners, allIds, imsize = read_chessboards(images, list(board_size))

Processing image test2/IMG_20240519_002830.jpg
Processing image test2/IMG_20240519_002849.jpg
Processing image test2/IMG_20240519_002846.jpg
Processing image test2/IMG_20240519_002831.jpg
Processing image test2/IMG_20240519_002836.jpg
Processing image test2/IMG_20240519_002828.jpg
Processing image test2/IMG_20240519_002848.jpg
Processing image test2/IMG_20240519_002842.jpg
Processing image test2/IMG_20240519_002832.jpg
Processing image test2/IMG_20240519_002839.jpg
Processing image test2/IMG_20240519_002845.jpg
Processing image test2/IMG_20240519_002835.jpg
Processing image test2/IMG_20240519_002833.jpg
Processing image test2/IMG_20240519_002844.jpg
Processing image test2/IMG_20240519_002841.jpg
Processing image test2/IMG_20240519_002838.jpg
Processing image test2/IMG_20240519_002829.jpg
Processing image test2/IMG_20240519_002840.jpg
Processing image test2/IMG_20240519_002843.jpg
Processing image test2/IMG_20240519_002834.jpg
Processing image test2/IMG_20240519_002837.jpg
Processing im

In [29]:
def calibrate_camera(allCorners, allIds, board, imsize):
    cameraMatrixInit = np.array([[ 2000.,    0., imsize[0]/2.],
                                 [    0., 2000., imsize[1]/2.],
                                 [    0.,    0.,           1.]])

    distCoeffsInit = np.zeros((4,1))
    flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL)
    
    (ret, camera_matrix, distortion_coefficients0,
     rotation_vectors, translation_vectors,
     stdDeviationsIntrinsics, stdDeviationsExtrinsics,
     perViewErrors) = cv2.aruco.calibrateCameraCharucoExtended(
                      charucoCorners=allCorners,
                      charucoIds=allIds,
                      board=board,
                      imageSize=imsize,
                      cameraMatrix=cameraMatrixInit,
                      distCoeffs=distCoeffsInit)

    return ret, camera_matrix, distortion_coefficients0, rotation_vectors, translation_vectors

dictionary = cv2.aruco.getPredefinedDictionary(arucoDict)
board = cv2.aruco.CharucoBoard(board_size, 4.0, 2.0, dictionary)

ret, mtx, dist, rvecs, tvecs = calibrate_camera(allCorners,allIds, board, imsize[1::-1])


In [30]:
data = {'mtx': mtx, 'dist': dist, 'rvecs': rvecs, 'tvecs': tvecs}
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [33]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
    
frame = cv2.imread(r'test/IMG_20240519_001635.jpg')
frame = cv2.undistort(frame, data['mtx'], data['dist'], None, data['mtx'])
cv2.imwrite('test.jpg', frame)



True

In [37]:
def detectCharucoBoardWithCalibrationPose(frame, boardSize, mtx, dist, rvec, tvec):
    dictionary = cv2.aruco.getPredefinedDictionary(arucoDict)
    board = cv2.aruco.CharucoBoard(boardSize, 0.04, 0.02, dictionary)
    params =  cv2.aruco.DetectorParameters()
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_NONE
    image_copy = frame.copy()
    charucodetector = cv2.aruco.CharucoDetector(board)
    charuco_corners, charuco_ids, marker_corners, marker_ids = charucodetector.detectBoard(image_copy)
    valid, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, dist, None, None)
    if valid: cv2.drawFrameAxes(image_copy, mtx, dist, rvec, tvec, length=0.1, thickness=15)
    cv2.imwrite('test.jpg', image_copy)
    return rvec, tvec

frame = cv2.imread(r'test2/IMG_20240519_002830.jpg')
detectCharucoBoardWithCalibrationPose(frame, board_size, data['mtx'], data['dist'], data['rvecs'], data['tvecs'])
    

(array([[-0.45081635],
        [ 0.26701475],
        [-2.69857934]]),
 array([[0.02803626],
        [0.0323735 ],
        [0.57622951]]))